In [1]:
print('Initializing data engineering!')
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import json, requests

Initializing data engineering!


### World data engineering
#### Fetching worldwide data

In [2]:
# df = pd.read_json('https://pomber.github.io/covid19/timeseries.json')
# df = pd.read_json('https://covidapi.info/api/v1/country/BRA')
# df = pd.read_json('https://api.covid19api.com/dayone/country/brazil/status/confirmed')
#https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest#cc76052f-6601-4645-80e5-ca7aaa36f8ef
df_countries = pd.read_csv('../data/world_countries_2019.csv')

df = pd.DataFrame()
url = "https://pomber.github.io/covid19/timeseries.json"
req = requests.get(url)
# r = r.json()
j = json.loads(req.text)

#### Fetching countries's pandemic data from Pomber's JSON to a dataframe 

In [3]:
# Loading countries names to dict
countries = []
df = pd.DataFrame()
for country in j:
    countries.append(country)

df['country'] = pd.Series(countries)

# Loading countries data do dict then to dataframe
dic = []
for country in countries:
    i = 0
    while i < len(j[country]):
        if j[country][i]['confirmed'] == 0:
            i += 1
            continue
        row = {'country': country, 'date': j[country][i]['date'], 'cases':j[country][i]['confirmed'],
               'deaths':j[country][i]['deaths'], 'recoveries':j[country][i]['recovered']}
        dic.append(row)
        i += 1 
df = pd.DataFrame.from_dict(dic)
df[df['country']=='France'].tail()

,country,date,cases,deaths,recoveries
3132,France,2020-4-21,158168,20829,39819
3133,France,2020-4-22,155980,21373,41326
3134,France,2020-4-23,158303,21889,42762
3135,France,2020-4-24,159952,22279,44271
3136,France,2020-4-25,161644,22648,45372


#### Feature engineering

In [4]:
for country in countries:
    qtdeDays = len(df[df.country == country])+1
    df.loc[df.country == country, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.country == country, 'case_day'] = df[df.country == country]['cases'].diff()    
    df.loc[df.country == country, 'death_day'] = df[df.country == country]['deaths'].diff()
    df.loc[df.country == country, 'recovery_day'] = df[df.country == country]['recoveries'].diff()

    # % daily variations
    df.loc[df.country == country, '%var_case_day'] = ((df[df.country == country]['case_day'] - df[df.country == country]['case_day'].shift()) / df[df.country == country]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_death_day'] = ((df[df.country == country]['death_day'] - df[df.country == country]['death_day'].shift()) / df[df.country == country]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_recovery_day'] = ((df[df.country == country]['recovery_day'] - df[df.country == country]['recovery_day'].shift()) / df[df.country == country]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.country == country) & (df.day == 1), 'case_day']= df.loc[(df.country == country) & (df.day==1), 'cases']
    df.loc[(df.country == country) & (df.day == 1), 'death_day']= df.loc[(df.country == country) & (df.day==1), 'deaths']
    df.loc[(df.country == country) & (df.day == 1), 'recovery_day']= df.loc[(df.country == country) & (df.day==1), 'recoveries']
    
    # Buscando a população do país e calculado os indicador per milhão
    if not df_countries[df_countries['country']==country].empty:
        million = df_countries[df_countries['country']==country]['PopTotal'].values[0] / 1000
        cases_million = (df[df.country == country]['case_day'] / million).round(1)
        deaths_million = (df[df.country == country]['death_day'] / million).round(1)
        recoveries_million = (df[df.country == country]['recovery_day'] / million).round(1)
    else:
        cases_million = 0
        deaths_million = 0
        recoveries_million = 0
        
    df.loc[df.country == country, 'cases_million'] = cases_million
    df.loc[df.country == country, 'deaths_million'] = deaths_million
    df.loc[df.country == country, 'recoveries_million'] = recoveries_million
    
    # moving averages (from last 7 days)
    df.loc[df.country == country, 'avg7_cases'] = df[df.country == country]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_deaths'] = df[df.country == country]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_recoveries'] = df[df.country == country]['recovery_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    
    df.loc[df.country == country, 'avg7_cases_million'] = df[df.country == country]['cases_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_deaths_million'] = df[df.country == country]['deaths_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_recoveries_million'] = df[df.country == country]['recoveries_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,country,date,cases,deaths,recoveries,day,case_day,death_day,recovery_day,%var_case_day,...,recoveries_million,avg7_cases,avg7_deaths,avg7_recoveries,avg7_cases_million,avg7_deaths_million,avg7_recoveries_million,perc_death,perc_recovery,active_cases
9823,Yemen,2020-4-21,1,0,0,12,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9824,Yemen,2020-4-22,1,0,0,13,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9825,Yemen,2020-4-23,1,0,0,14,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9826,Yemen,2020-4-24,1,0,1,15,0,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0
9827,Yemen,2020-4-25,1,0,1,16,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0


#### Saving CSV

In [5]:
df.to_csv('../data/world_corona19_data.csv', index = False)

In [6]:
df[df['country']=='France'].tail()

,country,date,cases,deaths,recoveries,day,case_day,death_day,recovery_day,%var_case_day,...,recoveries_million,avg7_cases,avg7_deaths,avg7_recoveries,avg7_cases_million,avg7_deaths_million,avg7_recoveries_million,perc_death,perc_recovery,active_cases
3132,France,2020-4-21,158168,20829,39819,89,2775,537,1783,16.50,...,27.4,3971.0,725.0,1531.0,60.0,11.0,23.0,13.17,25.18,97520
3133,France,2020-4-22,155980,21373,41326,90,-2188,544,1507,-178.85,...,23.1,3199.0,597.0,1408.0,49.0,9.0,21.0,13.70,26.49,93281
3134,France,2020-4-23,158303,21889,42762,91,2323,516,1436,-206.17,...,22.0,1746.0,564.0,1347.0,26.0,8.0,20.0,13.83,27.01,93652
3135,France,2020-4-24,159952,22279,44271,92,1649,390,1509,-29.01,...,23.2,1695.0,510.0,1323.0,26.0,7.0,20.0,13.93,27.68,93402
3136,France,2020-4-25,161644,22648,45372,93,1692,369,1101,2.61,...,16.9,1936.0,471.0,1255.0,29.0,7.0,19.0,14.01,28.07,93624


#### countries not located in UN dataset

In [7]:
for country in countries:
    if df_countries[df_countries['country']==country]['PopTotal'].empty:
        print(country)
# df[df['pais'] == pais].tail()
# df_countries[df_countries['Location']==pais]['PopTotal']

Bosnia and Herzegovina
Brunei
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Diamond Princess
Eswatini
Holy See
Iran
Korea, South
Moldova
North Macedonia
Russia
Taiwan*
Tanzania
United Arab Emirates
Venezuela
Vietnam
Syria
Laos
Libya
West Bank and Gaza
Kosovo
Burma
MS Zaandam


### Brazil data engineering

In [8]:
df = pd.read_csv('../data/gov_brazil_corona19_data.csv', sep=';')
df.rename(columns={'regiao': 'region', 'estado':'state', 'data':'date','casosNovos': 'case_day', 'casosAcumulados':'cases', 'obitosNovos':'death_day','obitosAcumulados':'deaths'}, inplace= True)
df['date'] = df['date'].astype('datetime64[ns]')

df.tail()

,region,state,date,case_day,cases,death_day,deaths
2344,Centro-Oeste,DF,2020-04-21,9,881,0,24
2345,Centro-Oeste,DF,2020-04-22,65,946,1,25
2346,Centro-Oeste,DF,2020-04-23,17,963,0,25
2347,Centro-Oeste,DF,2020-04-24,26,989,1,26
2348,Centro-Oeste,DF,2020-04-25,24,1013,0,26


#### Feature engineering

In [9]:
states = df.state.unique()
df.drop(df[df['cases'] == 0 ].index, axis=0, inplace= True)

for state in states:
    qtdeDays = len(df[df.state == state])+1
    df.loc[df.state == state, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.state == state, 'case_day'] = df[df.state == state]['cases'].diff()    
    df.loc[df.state == state, 'death_day'] = df[df.state == state]['deaths'].diff()
#     df.loc[df.state == state, 'recovery_day'] = df[df.state == state]['recoveries'].diff()

    # % daily variations
    df.loc[df.state == state, '%var_case_day'] = ((df[df.state == state]['case_day'] - df[df.state == state]['case_day'].shift()) / df[df.state == state]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.state == state, '%var_death_day'] = ((df[df.state == state]['death_day'] - df[df.state == state]['death_day'].shift()) / df[df.state == state]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
#     df.loc[df.state == state, '%var_recovery_day'] = ((df[df.state == state]['recovery_day'] - df[df.state == state]['recovery_day'].shift()) / df[df.state == state]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.state == state) & (df.day == 1), 'case_day']= df.loc[(df.state == state) & (df.day==1), 'cases']
    df.loc[(df.state == state) & (df.day == 1), 'death_day']= df.loc[(df.state == state) & (df.day==1), 'deaths']
    
    # moving averages (from last 7 days)
    df.loc[df.state == state, 'avg7_cases'] = df[df.state == state]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.state == state, 'avg7_deaths'] = df[df.state == state]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.state == state, 'perc_death'] = (df[df.state == state]['deaths']/df[df.state == state]['cases']*100).round(2) 
    df.loc[df.state == state, 'avg7_perc_death'] = df[df.state == state]['perc_death'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
# df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
# df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
# df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,region,state,date,case_day,cases,death_day,deaths,day,%var_case_day,%var_death_day,avg7_cases,avg7_deaths,perc_death,avg7_perc_death
2344,Centro-Oeste,DF,2020-04-21,9,881,0,24,46,-80.00,0.0,32.0,1.0,2.72,2.78
2345,Centro-Oeste,DF,2020-04-22,65,946,1,25,47,622.22,0.0,37.0,1.0,2.64,2.80
2346,Centro-Oeste,DF,2020-04-23,17,963,0,25,48,-73.85,-100.0,35.0,0.0,2.60,2.78
2347,Centro-Oeste,DF,2020-04-24,26,989,1,26,49,52.94,0.0,34.0,0.0,2.63,2.77
2348,Centro-Oeste,DF,2020-04-25,24,1013,0,26,50,-7.69,-100.0,35.0,0.0,2.57,2.69


In [10]:
df.to_csv('../data/brazil_corona19_data.csv', index = False)

### São Paulo data engineering

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
df.rename(columns={'munic': 'city', 'casos': 'cases','obitos':'deaths'}, inplace= True)
df['date'] = pd.to_datetime(dict(year=2020, month=df.mes, day=df.dia))
df.fillna(0, inplace = True)
df.drop(['dia','mes'], inplace=True, axis=1)

df = df[~df['city'].isin(['ignorado','outro estado', 'total geral', 'outro pais'])] 

df.tail()

,city,cases,deaths,codigo_ibge,latitude,longitude,date
4233,varzea paulista,8.0,2.0,3556503.0,-23.2136,-46.8234,2020-04-24
4234,vinhedo,19.0,1.0,3556701.0,-23.0302,-46.9833,2020-04-24
4235,vista alegre do alto,1.0,0.0,3556909.0,-21.1692,-48.6284,2020-04-24
4236,votorantim,8.0,2.0,3557006.0,-23.5446,-47.4388,2020-04-24
4237,votuporanga,9.0,0.0,3557105.0,-20.4237,-49.9781,2020-04-24


#### Feature engineering

In [12]:
cities = df.city.unique()
df.drop(df[df['cases'] == 0 ].index, axis=0, inplace= True)

for city in cities:
    qtdeDays = len(df[df.city == city])+1
    df.loc[df.city == city, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.city == city, 'case_day'] = df[df.city == city]['cases'].diff()    
    df.loc[df.city == city, 'death_day'] = df[df.city == city]['deaths'].diff()
#     df.loc[df.city == city, 'recovery_day'] = df[df.city == city]['recoveries'].diff()

    # % daily variations
    df.loc[df.city == city, '%var_case_day'] = ((df[df.city == city]['case_day'] - df[df.city == city]['case_day'].shift()) / df[df.city == city]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.city == city, '%var_death_day'] = ((df[df.city == city]['death_day'] - df[df.city == city]['death_day'].shift()) / df[df.city == city]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
#     df.loc[df.city == city, '%var_recovery_day'] = ((df[df.city == city]['recovery_day'] - df[df.city == city]['recovery_day'].shift()) / df[df.city == city]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.city == city) & (df.day == 1), 'case_day']= df.loc[(df.city == city) & (df.day==1), 'cases']
    df.loc[(df.city == city) & (df.day == 1), 'death_day']= df.loc[(df.city == city) & (df.day==1), 'deaths']
    
    # moving averages (from last 7 days)
    df.loc[df.city == city, 'avg7_cases'] = df[df.city == city]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.city == city, 'avg7_deaths'] = df[df.city == city]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.city == city, 'perc_death'] = (df[df.city == city]['deaths']/df[df.city == city]['cases']*100).round(2) 
    df.loc[df.city == city, 'avg7_perc_death'] = df[df.city == city]['perc_death'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
# df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
# df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
# df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,city,cases,deaths,codigo_ibge,latitude,longitude,date,day,case_day,death_day,%var_case_day,%var_death_day,avg7_cases,avg7_deaths,perc_death,avg7_perc_death
4237,votuporanga,9.0,0.0,3557105.0,-20.4237,-49.9781,2020-04-24,23,0,0,0.00,0.00,0.0,0.0,0.00,0.00
4238,ignorado,7.0,0.0,0.0,0.0000,0.0000,2020-04-24,25,0,0,0.00,0.00,0.0,0.0,0.00,0.00
4239,outro estado,14.0,0.0,0.0,0.0000,0.0000,2020-04-24,13,0,0,0.00,0.00,0.0,0.0,0.00,0.00
4240,outro pais,42.0,0.0,0.0,0.0000,0.0000,2020-04-24,12,0,0,0.00,0.00,0.0,0.0,0.00,0.00
4241,total geral,17826.0,1512.0,0.0,0.0000,0.0000,2020-04-24,9,8455,817,1676.26,839.08,2045.0,189.0,8.48,7.01


In [13]:
df.to_csv('../data/saoPaulo_corona19_data.csv', index = False)

In [14]:
# df[df['country']=='Belgium']
print('Data engineering done!')

Data engineering done!
